In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
all_library_info = pd.read_csv('../data3/interim/all_library_info.csv')
novel_cluster_preds = pd.read_csv('../data3/interim/ecor_unique_novel_proteins.csv')

In [3]:
ecor_seq_assemblies = pd.read_csv('../data/interim/ecor_seq_assemblies.csv', names=['product_accession', 'seq_id', 'seq', 'assembly'])
ecor_seqs = ecor_seq_assemblies[['product_accession', 'seq']].drop_duplicates()

In [4]:
long_library_info = all_library_info.copy()
long_library_info['product_accession'] = long_library_info['product_accessions'].str.split(', ')
long_library_info = long_library_info.explode('product_accession')
long_library_info = long_library_info[['product_accession', 'genomic_accession', 'working_id']]
long_library_info = (long_library_info.merge(ecor_seqs, how='inner', on='product_accession'))

In [5]:
with open('../data3/interim/ecor_cloned_seqs.faa', 'w') as f:
    for _, row in long_library_info.iterrows():
        print('>' + row['product_accession'], file=f)
        print(row['seq'], file=f)

In [6]:
cloned_accessions = long_library_info['product_accession'].to_list()

## Run HHPred

In [7]:
faa_file = '../data3/interim/ecor_cloned_seqs.faa'
msa_out_dir = '../data3/interim/ecor_cloned_msas/'
raw_out_dir ='../data3/interim/ecor_cloned_df_alignments/'
parsed_out_file='../data3/interim/ecor_cloned_df_out.csv'

In [11]:
if not os.path.exists(msa_out_dir):
    os.mkdir(msa_out_dir)

In [18]:
old_msa_dir = '../data3/interim/ecor_unique_plus_cloned_msas/'
for msa_f in tqdm(os.listdir(old_msa_dir)):
    if msa_f.split('.a')[0] in cloned_accessions:
        os.system(f"cp {old_msa_dir + msa_f} {msa_out_dir}")

100%|██████████| 11303/11303 [00:16<00:00, 684.03it/s]


In [19]:
os.system(' '.join(['conda run -n hhpred python', 
                        '~/Documents/hhpred/code/hhpred.py', 
                        '--raw_out_dir', raw_out_dir,  
                        '--parsed_out_file', parsed_out_file, 
                        '--dbs df2', 
                        '--in_fasta', faa_file,  
                        '--n_iter 3', 
                        '--alignment_dir', msa_out_dir, 
                        '--n_jobs 40']))

Building multiple sequence alignments from fasta
Wirting out individual fasta files to ~/Documents/hhpred/data/temp/temp_fasta/
Transferring the Uniclust database to a the local filesystem
Wirting out MSAs from uniclust to ../data3/interim/ecor_cloned_msas/
Reusing MSAs
Generating multiple sequence alignments with 3 iterations of HHblits
Querying df2 database(s)



0

## Run ESMFold + Foldseek

In [8]:
struct_out_dir = '../data3/interim/ecor_cloned_structures/'
df_rep_struct_db = '../data3/interim/foldseek_dbs/df_rep_foldseek_db_pad'
struct_db = '../data3/interim/foldseek_dbs/ecor_cloned_struct_db'
pad_struct_db = '../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad'
res_db = '../data3/interim/foldseek_dbs/ecor_cloned_df_rep_search'
struct_out_file = '../data3/interim/ecor_cloned_df_struct_search.txt'
struct_out_html = '../data3/interim/ecor_cloned_df_struct_search.html'

In [8]:
old_struct_dir = '../data3/interim/ecor_validated_structures/'

In [34]:
folded_accs = list()
for struct_f in os.listdir(old_struct_dir):
    if struct_f.split('.p')[0] in cloned_accessions:
        os.system(f"cp {old_struct_dir + struct_f} {struct_out_dir + struct_f}")
        folded_accs.append(struct_f.split('.p')[0])

In [9]:
struct_faa_file = '../data3/interim/ecor_cloned_seqs_missing_fold.faa'

In [24]:
filtered_library_info = long_library_info[~long_library_info['product_accession'].isin(folded_accs)]
with open(struct_faa_file, 'w') as f:
    for _, row in filtered_library_info.iterrows():
        print('>' + row['product_accession'], file=f)
        print(row['seq'], file=f)

In [35]:
os.system(' '.join(['conda run -n esmfold3 python', 
                    'fold.py', 
                    '-i', struct_faa_file, 
                    '-o', struct_out_dir]))

11.7
25/08/20 10:56:07 | INFO | root | Reading sequences from ../data3/interim/ecor_cloned_seqs_missing_fold.faa
25/08/20 10:56:07 | INFO | root | Loaded 104 sequences from ../data3/interim/ecor_cloned_seqs_missing_fold.faa
25/08/20 10:56:07 | INFO | root | Loading model
25/08/20 10:56:11 | INFO | torch.distributed.nn.jit.instantiator | Created a temporary directory at /state/partition1/slurm_tmp/2690206.0.0/tmphrn0xwe4
25/08/20 10:56:11 | INFO | torch.distributed.nn.jit.instantiator | Writing /state/partition1/slurm_tmp/2690206.0.0/tmphrn0xwe4/_remote_module_non_scriptable.py
25/08/20 10:57:05 | INFO | root | Starting Predictions
25/08/20 10:57:37 | INFO | root | Predicted structure for WP_252151311.1 with length 61, pLDDT 51.9, pTM 0.352 in 2.7s (amortized, batch size 12). 1 / 104 completed.
25/08/20 10:57:37 | INFO | root | Predicted structure for WP_001154434.1 with length 62, pLDDT 80.7, pTM 0.654 in 2.7s (amortized, batch size 12). 2 / 104 completed.
25/08/20 10:57:37 | INFO | ro

0

In [36]:
foldseek_preamble = ['conda run -n nvcc /home/gridsan/pdeweirdt/Documents/foldseek/build/bin/foldseek']

In [37]:
os.system(' '.join(foldseek_preamble + 
                   ['createdb', struct_out_dir, struct_db]))

createdb ../data3/interim/ecor_cloned_structures/ ../data3/interim/foldseek_dbs/ecor_cloned_struct_db 

MMseqs Version:             	e3fadcd07f971e864c094ac4f3a78bf4ed845e07
Use GPU                     	0
Path to ProstT5             	
Chain name mode             	0
Model name mode             	0
Createdb extraction mode    	0
Interface distance threshold	8
Write mapping file          	0
Mask b-factor threshold     	0
Coord store mode            	2
Write lookup file           	1
Input format                	0
File Inclusion Regex        	.*
File Exclusion Regex        	^$
Threads                     	80
Verbosity                   	3

Output file: ../data3/interim/foldseek_dbs/ecor_cloned_struct_db
=[================================================================] 176 0s 85ms
Time for merging to ecor_cloned_struct_db_ss: 0h 0m 0s 698ms
Time for merging to ecor_cloned_struct_db_h: 0h 0m 0s 530ms
Time for merging to ecor_cloned_struct_db_ca: 0h 0m 0s 406ms
Time for merging to ecor_cloned

0

In [38]:
os.system(' '.join(foldseek_preamble +
                   ['makepaddedseqdb', struct_db, pad_struct_db]))

makepaddedseqdb ../data3/interim/foldseek_dbs/ecor_cloned_struct_db ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad 

MMseqs Version:          	e3fadcd07f971e864c094ac4f3a78bf4ed845e07
Substitution matrix      	aa:3di.out,nucl:3di.out
Mask residues            	0
Mask residues probability	0.999995
Write lookup file        	1
Threads                  	80
Verbosity                	3
Cluster search           	0

lndb ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_h ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad_tmp_ss_h 

Time for processing: 0h 0m 0s 4ms
lndb ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_ss ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad_tmp_ss 

Time for processing: 0h 0m 0s 5ms
makepaddedseqdb ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad_tmp_ss ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad_ss --sub-mat 'aa:3di.out,nucl:3di.out' --score-bias 0 --mask 0 --mask-prob 0.999995 --mask-lower-case 1 --mask-n-repeat 6 --wr

0

In [39]:
os.system(' '.join(foldseek_preamble +
                   ['search', pad_struct_db, df_rep_struct_db, 
                    res_db, '$TMPDIR', 
                    '--gpu 1', '--alignment-type 1', '-a']))

search ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad ../data3/interim/foldseek_dbs/df_rep_foldseek_db_pad ../data3/interim/foldseek_dbs/ecor_cloned_df_rep_search /state/partition1/slurm_tmp/2690206.0.0 --gpu 1 --alignment-type 1 -a 

MMseqs Version:                    	e3fadcd07f971e864c094ac4f3a78bf4ed845e07
TMscore threshold                  	0
TMscore threshold mode             	0
LDDT threshold                     	0
Sort by structure bit score        	1
Alignment type                     	1
Exact TMscore                      	0
Substitution matrix                	aa:3di.out,nucl:3di.out
Add backtrace                      	true
Alignment mode                     	3
Alignment mode                     	0
E-value threshold                  	10
Seq. id. threshold                 	0
Min alignment length               	0
Seq. id. mode                      	0
Alternative alignments             	0
Coverage threshold                 	0
Coverage mode                      	0
Max seq

0

In [40]:
os.system(' '.join(foldseek_preamble +
                   ['convertalis', pad_struct_db, df_rep_struct_db, 
                    res_db, struct_out_file, 
                    '--format-output', 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,qcov,tcov,evalue,bits,alntmscore,qtmscore,ttmscore,rmsd,lddt,prob']))

convertalis ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad ../data3/interim/foldseek_dbs/df_rep_foldseek_db_pad ../data3/interim/foldseek_dbs/ecor_cloned_df_rep_search ../data3/interim/ecor_cloned_df_struct_search.txt --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,qcov,tcov,evalue,bits,alntmscore,qtmscore,ttmscore,rmsd,lddt,prob 

MMseqs Version:        	e3fadcd07f971e864c094ac4f3a78bf4ed845e07
Substitution matrix    	aa:3di.out,nucl:3di.out
Alignment format       	0
Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,qcov,tcov,evalue,bits,alntmscore,qtmscore,ttmscore,rmsd,lddt,prob
Gap open cost          	aa:10,nucl:10
Gap extension cost     	aa:1,nucl:1
Database output        	false
Preload mode           	0
Threads                	80
Compressed             	0
Verbosity              	3
Exact TMscore          	0

[=================================================================] 176 5s 701ms
Time fo

0

In [41]:
os.system(' '.join(foldseek_preamble +
                   ['convertalis', pad_struct_db, df_rep_struct_db, 
                    res_db, struct_out_html, 
                    '--format-mode 3']))

convertalis ../data3/interim/foldseek_dbs/ecor_cloned_struct_db_pad ../data3/interim/foldseek_dbs/df_rep_foldseek_db_pad ../data3/interim/foldseek_dbs/ecor_cloned_df_rep_search ../data3/interim/ecor_cloned_df_struct_search.html --format-mode 3 

MMseqs Version:        	e3fadcd07f971e864c094ac4f3a78bf4ed845e07
Substitution matrix    	aa:3di.out,nucl:3di.out
Alignment format       	3
Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
Gap open cost          	aa:10,nucl:10
Gap extension cost     	aa:1,nucl:1
Database output        	false
Preload mode           	0
Threads                	80
Compressed             	0
Verbosity              	3
Exact TMscore          	0

[=================================================================] 176 0s 691ms
Time for merging to ecor_cloned_df_struct_search.html: 0h 0m 0s 818ms
Time for processing: 0h 0m 2s 536ms



0

## Analyze homology results

In [9]:
hhpred_results = pd.read_csv(parsed_out_file)
foldseek_results = pd.read_table(struct_out_file,
                                 names=["query", "target", "fident", "alnlen", "mismatch", "gapopen", "qstart", "qend", "tstart", "tend", "qcov", "tcov", "evalue", "bits", "alntmscore", "qtmscore", "ttmscore", "rmsd", "lddt", "prob"])
predictions = pd.read_csv('../data3/interim/ecor_predictions_defensive_cat2.csv')
hit_seq_info = pd.read_csv('../data3/interim/hit_seq_info.csv')
display_name_df = pd.read_csv('../data/raw/Beaker validation domains - system_display_names.csv')
df_rep_info = pd.read_csv('../data/interim/df_top_uniprot_hits.csv')

/tmp/ipykernel_1356294/921762995.py:4: DtypeWarning: Columns (24,30,35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  predictions = pd.read_csv('../data3/interim/ecor_predictions_defensive_cat2.csv')


In [10]:
hit_categories = ['Defense homolog in expected system',
                  'Defense homolog in new context', 
                  'Remote defense homolog', 
                  'Structural defense homolog',
                  'Predicted novel defense gene', 
                  'Not defensive']
predictions['hit_category'] = pd.Categorical(predictions['hit_category'], 
                                             categories=hit_categories)

In [11]:
non_redundant_predictions = (predictions
                             .sort_values(['hit_category', 'mean_log_odds'], ascending=[True, False])
                             .groupby('cluster_id')
                             .head(1))

In [12]:
hhpred_results['q_ali_len'] = hhpred_results['qend'] - hhpred_results['qstart']
hhpred_homologs = (hhpred_results[(hhpred_results['Prob'] > 50)]
                           .sort_values('q_ali_len', ascending=False)
                           .groupby('query')
                           .head(1))

In [13]:
foldseek_homologs = (foldseek_results[foldseek_results['prob'] > 0.6]
                     .sort_values('prob', ascending=False)
                     .groupby('query')
                     .head(1))

**Assigning validated hits**

If hit is part of cluster that is not not defensive:

    Assign hit to category

Elif hit has log-odds > 0:
    
    Assign hit to category based on hhpred and foldseek results

Else:

    Assign hit to non-defensive category


In [14]:
def assign_hit_category(row, qcov=0.4):
    if row['mean_log_odds'] < 0:
        return 'Not defensive'
    elif row['cluster_hit_category'] != 'Not defensive':
        return row['cluster_hit_category']
    elif row['defense_system_protein']:
        return 'Defense homolog in expected system'
    elif row['defense_homolog']:
        return 'Defense homolog in new context'
    elif row['hhpred_qcov'] > qcov:
        return 'Remote defense homolog'
    elif row['foldseek_qcov'] > qcov:
        return 'Structural defense homolog'
    else:
        return 'Predicted novel defense gene'

In [15]:
merged_long_library_info = (long_library_info.merge(predictions[['product_accession', 'genomic_accession','cluster_id', 'defense_homolog', 
                                                                 'defense_system_protein', 'product_length', 'mean_log_odds', 'start', 'strand']], 
                                          how='inner', on=['product_accession', 'genomic_accession'])
                       .merge(non_redundant_predictions[['cluster_id', 'hit_category']]
                              .rename(columns={'hit_category': 'cluster_hit_category'}), 
                              how='inner', on='cluster_id')
                       .merge(display_name_df, how='left', on='working_id')
                       .merge(hhpred_homologs[['query', 'hit_name', 'q_ali_len']]
                              .rename(columns={'query': 'product_accession', 
                                               'hit_name': 'hhpred_hit_name', 
                                               'q_ali_len': 'hhpred_q_ali_len'}), 
                              how='left', on='product_accession')
                       .merge(foldseek_homologs[['query', 'target', 'qcov']]
                              .merge(df_rep_info[['target', 'description']], how='inner', on='target')
                              .rename(columns={'query': 'product_accession',
                                               'description': 'foldseek_target_homolog',
                                               'target': 'foldseek_target',
                                               'qcov': 'foldseek_qcov'}), 
                              how='left', on='product_accession'))
merged_long_library_info['fake_start'] = pd.Series([1 if x == '+' else -1 for x in merged_long_library_info['strand']]) * merged_long_library_info['start']
merged_long_library_info['protein_num'] = merged_long_library_info.groupby('working_id')['fake_start'].rank()
merged_long_library_info['hhpred_qcov'] = merged_long_library_info['hhpred_q_ali_len']/merged_long_library_info['product_length']
merged_long_library_info['hit_category'] = merged_long_library_info.apply(assign_hit_category, axis=1)
merged_long_library_info['hit_category'] = pd.Categorical(merged_long_library_info['hit_category'], categories=['Not defensive'] + hit_categories[:-1])
id_top_category = (merged_long_library_info.sort_values('hit_category', ascending=False)
                   .groupby('working_id')
                   .head(1)
                   [['working_id', 'hit_category', 'DS_name']]
                   .rename(columns={'hit_category':'top_hit_category'}))
merged_long_library_info = (merged_long_library_info.merge(id_top_category.drop(columns='DS_name'), how='inner', on='working_id'))

In [16]:
len(merged_long_library_info)

153

In [17]:
novel_cluster_preds.columns

Index(['protein_context_id', 'mean_log_odds', 'sd_log_odds', 'min_log_odds',
       'max_log_odds', '# feature', 'class', 'assembly', 'assembly_unit',
       'seq_type', 'chromosome', 'genomic_accession', 'start', 'end', 'strand',
       'product_accession', 'non-redundant_refseq', 'related_accession',
       'name', 'symbol', 'GeneID', 'locus_tag', 'feature_interval_length',
       'product_length', 'attributes', 'assembly_stub', 'protein_assembly',
       'defense_homolog', 'defense_system_protein', 'defense_homolog_names',
       'sys_id', 'operon', 'contig_end', 'cluster_id', 'MG1655_homolog',
       'hit_name', 'q_cov', 'Prob', 'hit_category', 'foldseek_hit', 'fident',
       'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend',
       'foldseek_q_cov', 'tcov', 'evalue', 'bits', 'alntmscore', 'qtmscore',
       'ttmscore', 'rmsd', 'lddt', 'foldseek_prob', 'accession_stub',
       'present', 'n_proteins'],
      dtype='object')

In [18]:
no_cov_novel_clusters = novel_cluster_preds.loc[novel_cluster_preds['q_cov'] == 0, 'product_accession']

In [19]:
no_cov_validated_clusters = (merged_long_library_info[~merged_long_library_info['DS_name'].isna() & 
                                                      merged_long_library_info['cluster_id'].isin(no_cov_novel_clusters)]
                             .reset_index(drop=True))
no_cov_validated_clusters['ds_num'] = no_cov_validated_clusters['DS_name'].str.split('-', expand=True)[1].astype(int)
print('# clusters:', no_cov_validated_clusters['cluster_id'].nunique())
print('# proteins', len(no_cov_validated_clusters))

# clusters: 18
# proteins 18


In [20]:
no_cov_validated_clusters.sort_values(['ds_num', 'protein_num'])[['DS_name', 'protein_num', 'hit_category']]

,DS_name,protein_num,hit_category
1,DS-1,1.0,Predicted novel defense gene
0,DS-1,2.0,Predicted novel defense gene
13,DS-2,3.0,Predicted novel defense gene
10,DS-3,1.0,Predicted novel defense gene
11,DS-4,1.0,Predicted novel defense gene
12,DS-4,5.0,Predicted novel defense gene
6,DS-5,2.0,Predicted novel defense gene
15,DS-6,2.0,Predicted novel defense gene
17,DS-9,1.0,Predicted novel defense gene
2,DS-13,2.0,Predicted novel defense gene


In [21]:
(~merged_long_library_info['DS_name'].isna() & (merged_long_library_info['cluster_id'].isin(no_cov_novel_clusters)))

0      False
1      False
2      False
3      False
4      False
       ...  
148    False
149    False
150    False
151    False
152    False
Length: 153, dtype: bool

In [22]:
((merged_long_library_info['hit_category'] == 'Predicted novel defense gene') & 
 merged_long_library_info['hhpred_q_ali_len'].isna() & 
 ~merged_long_library_info['DS_name'].isna()).sum()


28

In [23]:
id_top_category['top_hit_category'].value_counts()

top_hit_category
Predicted novel defense gene          73
Remote defense homolog                13
Structural defense homolog             8
Not defensive                          0
Defense homolog in expected system     0
Defense homolog in new context         0
Name: count, dtype: int64

In [24]:
id_top_category.loc[~id_top_category['DS_name'].isna(), 'top_hit_category'].value_counts()

top_hit_category
Predicted novel defense gene          32
Remote defense homolog                 7
Structural defense homolog             3
Not defensive                          0
Defense homolog in expected system     0
Defense homolog in new context         0
Name: count, dtype: int64

In [26]:
highlight_hit_seq_info = merged_long_library_info[~merged_long_library_info['DS_name'].isna()].copy()
highlight_hit_seq_info['ds_num'] = highlight_hit_seq_info['DS_name'].str.split('-', expand=True)[1].astype(int)
highlight_hit_seq_info = (highlight_hit_seq_info
                          .sort_values(['ds_num', 'protein_num'])
                          .loc[highlight_hit_seq_info['hit_category'] == highlight_hit_seq_info['top_hit_category'], 
                                                  ['DS_name', 'protein_num', 'hit_category']])

In [27]:
highlight_hit_seq_info

,DS_name,protein_num,hit_category
10,DS-1,1.0,Predicted novel defense gene
9,DS-1,2.0,Predicted novel defense gene
92,DS-2,3.0,Predicted novel defense gene
70,DS-3,1.0,Predicted novel defense gene
72,DS-4,1.0,Predicted novel defense gene
74,DS-4,3.0,Predicted novel defense gene
76,DS-4,5.0,Predicted novel defense gene
29,DS-5,1.0,Predicted novel defense gene
30,DS-5,2.0,Predicted novel defense gene
97,DS-6,1.0,Predicted novel defense gene


In [28]:
id_top_category['top_hit_category'].value_counts()

top_hit_category
Predicted novel defense gene          73
Remote defense homolog                13
Structural defense homolog             8
Not defensive                          0
Defense homolog in expected system     0
Defense homolog in new context         0
Name: count, dtype: int64